In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on
# %pycodestyle_off -to turn it off

In [2]:
# import all libraries
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
import requests
import json
import pandas as pd
from binance import Client
from datetime import datetime
import os
from dotenv import load_dotenv, dotenv_values

load_dotenv()

True

In [3]:
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
MINIO_USER = os.getenv("MINIO_ROOT_USER")
MINIO_PASSWORD = os.getenv("MINIO_ROOT_PASSWORD")
client_binance = Client(API_KEY, SECRET_KEY)

In [4]:
spark = SparkSession.builder \
    .appName("CryptoETL") \
    .getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_USER)#turn into access key in the future 
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_PASSWORD)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")


'''
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")'''


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 14:05:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/02 14:05:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


'\nsc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")\nsc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")\nsc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\nsc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")'

6:67: E261 at least two spaces before inline comment
6:67: E262 inline comment should start with '# '
6:80: E501 line too long (101 > 79 characters)
6:102: W291 trailing whitespace
12:80: E501 line too long (80 > 79 characters)
19:80: E501 line too long (90 > 79 characters)
21:1: W391 blank line at end of file


In [5]:
# tickers = client.get_all_tickers()
# client.get_exchange_info() #has rate limit info
# client.response.headers #has info on current usage

In [6]:
# start_datetime = datetime.strptime("01-01-2023", '%d-%m-%Y')
# start_date = start_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")

# end_datetime = datetime.strptime("01-01-2024", '%d-%m-%Y')
# end_date_date = end_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")

In [7]:
start_date = "01-01-2023"
end_date = "12-31-2023"
symbol = "BTCUSDT"
interval = "1M"

In [8]:
klines = client_binance.get_historical_klines(
    symbol=symbol, interval=interval, start_str=start_date, end_str=end_date, limit=1000
)

2:80: E501 line too long (88 > 79 characters)


In [9]:
# api response structure
"""[
  [
    1499040000000,      // Kline open time
    "0.01634790",       // Open price
    "0.80000000",       // High price
    "0.01575800",       // Low price
    "0.01577100",       // Close price
    "148976.11427815",  // Volume
    1499644799999,      // Kline Close time
    "2434.19055334",    // Quote asset volume
    308,                // Number of trades
    "1756.87402397",    // Taker buy base asset volume
    "28.46694368",      // Taker buy quote asset volume
    "0"                 // Unused field, ignore.
  ]
]"""

'[\n  [\n    1499040000000,      // Kline open time\n    "0.01634790",       // Open price\n    "0.80000000",       // High price\n    "0.01575800",       // Low price\n    "0.01577100",       // Close price\n    "148976.11427815",  // Volume\n    1499644799999,      // Kline Close time\n    "2434.19055334",    // Quote asset volume\n    308,                // Number of trades\n    "1756.87402397",    // Taker buy base asset volume\n    "28.46694368",      // Taker buy quote asset volume\n    "0"                 // Unused field, ignore.\n  ]\n]'

In [10]:
columns = [
    "Kline open time",
    "Open Price",
    "High price",
    "Low price",
    "Close Price",
    "Volume",
    "Kline Close time",
    "Quote asset volume",
    "Number of trades",
    "Taker buy base asset volume",
    "Taker buy quote asset volume",
    "Ignore",
]

In [11]:
dataframe = spark.createDataFrame(klines, columns)

In [12]:
df = dataframe.withColumn("Kline open time", to_timestamp(col("Kline open time") / 1000))
df = df.withColumnRenamed("Kline open time","datetime")
df = df.select('datetime','Open Price','Close Price','Volume')

1:80: E501 line too long (89 > 79 characters)
2:44: E231 missing whitespace after ','
3:26: E231 missing whitespace after ','
3:39: E231 missing whitespace after ','
3:53: E231 missing whitespace after ','


In [13]:
df.show()

+-------------------+--------------+--------------+----------------+
|           datetime|    Open Price|   Close Price|          Volume|
+-------------------+--------------+--------------+----------------+
|2023-01-01 00:00:00|16541.77000000|23125.13000000|7977028.87801000|
|2023-02-01 00:00:00|23125.13000000|23141.57000000|8642691.27165000|
|2023-03-01 00:00:00|23141.57000000|28465.36000000|9516189.35846000|
|2023-04-01 01:00:00|28465.36000000|29233.21000000|1626745.55850000|
|2023-05-01 01:00:00|29233.20000000|27210.35000000|1302000.49221000|
|2023-06-01 01:00:00|27210.36000000|30472.00000000|1387207.48275000|
|2023-07-01 01:00:00|30471.99000000|29232.25000000| 925773.81731000|
|2023-08-01 01:00:00|29232.26000000|25940.78000000|1025866.55023000|
|2023-09-01 01:00:00|25940.77000000|26962.56000000| 809329.04893000|
|2023-10-01 01:00:00|26962.57000000|34639.77000000|1141403.67990000|
|2023-11-01 00:00:00|34639.78000000|37723.96000000|1055690.59638000|
|2023-12-01 00:00:00|37723.9700000

In [14]:
from minio import Minio

client_minio = Minio(
    "localhost:9000",  # Make sure you're using port 9000 for the S3 API
    access_key = MINIO_USER,
    secret_key = MINIO_PASSWORD,
    secure=False  # Disable SSL if you're not using SSL certificates
)


5:15: E251 unexpected spaces around keyword / parameter equals
5:17: E251 unexpected spaces around keyword / parameter equals
6:15: E251 unexpected spaces around keyword / parameter equals
6:17: E251 unexpected spaces around keyword / parameter equals
9:1: W391 blank line at end of file


In [15]:
buckets = client_minio.list_buckets()

In [16]:
#df.write.csv('test',header = True)

1:1: E265 block comment should start with '# '


In [17]:
#df.coalesce(1).write.csv("csv", header=True)
destination_file = 'csv'
source_file = './csv/part-00000-403543db-3e05-4013-a90b-934dff0f9694-c000.csv' #just an object not technically a file
bucket_name = 'bucket1'

1:1: E265 block comment should start with '# '
3:79: E261 at least two spaces before inline comment
3:80: E262 inline comment should start with '# '
3:80: E501 line too long (117 > 79 characters)


In [ ]:
client_minio.fput_object(
        bucket_name, destination_file, source_file,
    )


In [18]:
df.write.format("parquet").mode('append').save(f"s3a://{bucket_name}/BTC")

25/03/02 14:05:23 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/03/02 14:05:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/02 14:05:29 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
